### Import Libraries 

In [ ]:
import pandas as pd
import sqlalchemy as sa
import json
from datetime import datetime

from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

from creds import creds

### Establish Connection to Database

In [ ]:
dest_engine = sa.create_engine(f"postgresql+psycopg2://{creds['user']}:{creds['pass']}@{creds['host']}:{creds['port']}/{creds['db']}")

### Read Target data from Database

In [ ]:
saham_dest = pd.read_sql_table('saham', dest_engine)
saham_dest

### Get Source Data

In [ ]:
def get_scrap_data(url):
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--window-size=1920x1080")
    user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'
    options.add_argument(f'user-agent={user_agent}')
    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    with driver as browser:
        # Menggunakan Selenium untuk membuka website
        browser.get(url)
        
        # Mengambil semua text yang ada di webpage
        content = browser.find_element(By.TAG_NAME, "pre").text

        # Mengubah text ke dalam bentuk json
        parsed_json = json.loads(content)
        
        return parsed_json

def getstocklistData():
    DaftarSaham = pd.DataFrame()
    
    URL = 'https://www.idx.co.id/primary/StockData/GetSecuritiesStock?start=0&length=9999&code=&sector=Energy&board=&language=id-id'
        
    data = get_scrap_data(URL)

    # Mengubah json ke dalam bentuk DataFrame
    df = pd.DataFrame(data['data'])

    # Mengubah format tanggal menjadi datetime
    df['ListingDate'] = [datetime.strptime(
        x[:10], '%Y-%m-%d') for x in df['ListingDate']]
    df['Sector'] = "Energy"

    DaftarSaham = pd.concat([DaftarSaham, df], ignore_index=True)

    # urut berdasarkan kode saham
    DaftarSaham = DaftarSaham.sort_values(by='Code').reset_index(drop=True)

    
    return DaftarSaham

saham_source = getstocklistData()
saham_source

### Get Changes (New & Updated Record)

In [ ]:
#Get all Changes
changes = saham_source[~saham_source.apply(tuple, 1).isin(saham_dest.apply(tuple, 1))]
changes

In [ ]:
#Get modified rows
modified = changes[changes.Code.isin(saham_dest.Code)]
modified

In [ ]:
## Get new rows
inserts = changes[~changes.Code.isin(saham_dest.Code)]
inserts

### Insert & Update

In [ ]:
#Insert
inserts.to_sql('saham', dest_engine, if_exists='append', index=False)

In [ ]:
#Update
def update_to_sql(df, table_name, key_name):
    a = []
    table = table_name
    primary_key = key_name
    temp_table = f"{table_name}_temporary_table"
    for col in df.columns:
        if col == primary_key:
            continue
        a.append(f'"{col}"=s."{col}"')
    df.to_sql(temp_table, dest_engine, if_exists='replace', index=False)
    update_stmt_1 = f'UPDATE public."{table}" f '
    update_stmt_2 = "SET "
    update_stmt_3 = ", ".join(a)
    update_stmt_4 = f' FROM public."{table}" t '
    update_stmt_5 = f' INNER JOIN (SELECT * FROM public."{temp_table}") AS s ON s."{primary_key}"=t."{primary_key}" '
    update_stmt_6 = f' Where f."{primary_key}"=s."{primary_key}" '
    update_stmt_7 = update_stmt_1 + update_stmt_2 + update_stmt_3 + update_stmt_4 + update_stmt_5 +  update_stmt_6 +";"
    print(update_stmt_7)
    with dest_engine.begin() as cnx:
        cnx.execute(update_stmt_7)


In [ ]:
update_to_sql(modified, 'saham', 'Code')